# Connect to Redshift

In [ ]:
import psycopg2

# Connect to Redshift and return connection object
def get_connection():
  # Connection information
  host = "default-workgroup.629479324349.us-west-2.redshift-serverless.amazonaws.com"
  port = "5439"  # Redshift port
  database = "dev"
  user = "admin"
  password = "*******"
  # Connect
  conn = psycopg2.connect(
      dbname=database,
      user=user,
      password=password,
      host=host,
      port=port
  )
  return conn

# Close the connection
def close(conn):
  conn.cursor().close()
  conn.close()

# Test connection

In [ ]:
# Connect
conn = get_connection()

# Get cursor
cur = conn.cursor()

# Test sql execution that list all users
cur.execute("SELECT * FROM pg_user;")
rows = cur.fetchall()

# Check the result
for row in rows: # print rows
    print(row)

# Disconnect
cur.close()
conn.close()

('rdsdb', 1, True, True, True, '********', 'infinity', None)
('IAM:RootIdentity', 101, True, True, False, '********', None, None)
('user1', 102, True, False, False, '********', None, None)
('admin', 100, True, True, False, '********', 'infinity', None)


#Initial load the dataset

## From S3 as external storage

In [ ]:
conn = get_connection()
cur = conn.cursor()

sql_create_external_schema = (
    "CREATE EXTERNAL SCHEMA external_schema_s3 "
    "FROM data catalog "
    "DATABASE 'external_db_s3' "
    "IAM_ROLE 'arn:aws:iam::629479324349:role/redshift-ETL-manager' "
    "CREATE EXTERNAL DATABASE IF NOT EXISTS;"
)

sql_create_external_table = (
    "CREATE EXTERNAL TABLE external_schema_s3.spotify_songs ("
      "track_id character varying(256),"
      "track_name character varying(256),"
      "track_artist character varying(256),"
      "track_popularity integer,"
      "track_album_id character varying(256),"
      "track_album_name character varying(256),"
      "track_album_release_date date,"
      "playlist_name character varying(256),"
      "playlist_id character varying(256), "
      "playlist_genre character varying(256),"
      "playlist_subgenre character varying(256),"
      "danceability real,"
      "energy real,"
      "key integer,"
      "loudness real,"
      "mode integer,"
      "speechiness real,"
      "acousticness real,"
      "instrumentalness real,"
      "liveness real,"
      "valence real,"
      "tempo real,"
      "duration_ms integer"
    ") "
    "ROW FORMAT DELIMITED "
    "FIELDS TERMINATED BY ',' "
    "LINES TERMINATED BY '\n' " # If skipped this line and set with '\n' by default
    "STORED AS textfile "
    "LOCATION 's3://practice-bucket50412/redshift_input/' "
    "TABLE PROPERTIES ('skip.header.line.count'='1');"

)

# Enable autocommit To resolve that SQL (CREATE EXTERNAL ...) cannot run inside a transaction block
conn.autocommit = True

# cur.execute(sql_create_external_schema)
cur.execute(sql_create_external_table)

close(conn)

# (Not be used) Alternative way to load csv file using COPY


In [ ]:
conn = get_connection()
cur = conn.cursor()

query_table_definition = (
  "CREATE TABLE raw_data.spotify_songs ("
    "track_id character varying(256) ENCODE lzo,"
    "track_name character varying(256) ENCODE lzo,"
    "track_artist character varying(256) ENCODE lzo,"
    "track_popularity integer ENCODE az64,"
    "track_album_id character varying(256) ENCODE lzo,"
    "track_album_name character varying(256) ENCODE lzo,"
    "track_album_release_date character varying(256) ENCODE lzo,"
    "playlist_name character varying(256) ENCODE lzo,"
    "playlist_id character varying(256) ENCODE lzo, "
    "playlist_genre character varying(256) ENCODE lzo,"
    "playlist_subgenre character varying(256) ENCODE lzo,"
    "danceability real ENCODE raw,"
    "energy real ENCODE raw,"
    "key integer ENCODE az64,"
    "loudness real ENCODE raw,"
    "mode integer ENCODE az64,"
    "speechiness real ENCODE raw,"
    "acousticness real ENCODE raw,"
    "instrumentalness real ENCODE raw,"
    "liveness real ENCODE raw,"
    "valence real ENCODE raw,"
    "tempo real ENCODE raw,"
    "duration_ms integer ENCODE az64"
  ")DISTKEY(playlist_id);"
)

sql_copy_csv = (
    "COPY dev.raw_data.spotify_songs "
    "FROM 's3://practice-bucket50412/spotify_songs.csv' "
    "IAM_ROLE 'arn:aws:iam::629479324349:role/redshift-ETL-manager' "
    "FORMAT AS CSV "
    "DELIMITER ',' "
    "QUOTE '\"' "
    "IGNOREHEADER 1 "
    "REGION AS 'us-west-2';"
)

cur.execute("BEGIN;")
cur.execute(sql_create_table)
cur.execute(sql_copy)
cur.execute("END;")

close(conn)

# Create tables to analysis

## Genres by year

In [ ]:
sql_create_table_genres_by_year = (
'CREATE TABLE dev.analytics.genre_by_year AS '
  'SELECT EXTRACT(YEAR FROM track_album_release_date) AS year, playlist_genre AS genre, COUNT(DISTINCT track_id) AS tracks '
  'FROM external_schema_s3.spotify_songs '
  'GROUP BY 1,2;'
)

conn = get_connection()
cur = conn.cursor()

cur.execute('BEGIN;')
cur.execute(sql_create_table_genres_by_year)
cur.execute('END;')

close(conn)

## Top 50 artists

In [ ]:
sql_create_table_tracks_by_artist = (
    'CREATE TABLE dev.analytics.tracks_by_artist AS '
      'SELECT '
        'track_artist AS artist, '
        'COUNT(DISTINCT track_id) AS tracks '
      'FROM external_schema_s3.spotify_songs '
      'GROUP BY track_artist'
)

conn = get_connection()
cur = conn.cursor()

cur.execute('BEGIN;')
cur.execute(sql_create_table_tracks_by_artist)
cur.execute('END;')

close(conn)

## t-SNE

### Load

In [ ]:
sql_create_external_table_tsne = (
  "CREATE EXTERNAL TABLE external_schema_s3.spotify_songs_tsne ("
      "playlist_genre character varying(256),"
      "dimension_1 real,"
      "dimension_2 real"
    ") "
    "ROW FORMAT DELIMITED "
    "FIELDS TERMINATED BY ',' "
    "LINES TERMINATED BY '\n' " # If skipped this line and set with '\n' by default
    "STORED AS textfile "
    "LOCATION 's3://practice-bucket50412/tsne/' "
    "TABLE PROPERTIES ('skip.header.line.count'='1');"
)

conn = get_connection()
cur = conn.cursor()

conn.autocommit = True
cur.execute(sql_create_external_table_tsne)


close(conn)

### Sample

In [ ]:
sql_create_table_tsne_sample = (
  'CREATE TABLE dev.analytics.spotify_songs_tsne_sampled AS '
    'WITH RankedData AS ( '
      'SELECT '
        'playlist_genre,'
        'dimension_1,'
        'dimension_2,'
        'ROW_NUMBER() OVER (PARTITION BY playlist_genre ORDER BY RAND()) AS row_num '
      'FROM '
        'dev.external_schema_s3.spotify_songs_tsne'
    ')'
    'SELECT '
      'playlist_genre,'
      'dimension_1,'
      'dimension_2 '
    'FROM '
      'RankedData '
    'WHERE '
      'row_num <= 100; '
)

conn = get_connection()
cur = conn.cursor()

cur.execute('BEGIN;')
cur.execute(sql_create_table_tsne_sample)
cur.execute('END;')

close(conn)